In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt
import folium

In [ ]:
!kaggle kaggle competitions download -c covid19-global-forecasting-week-1

In [ ]:
!pip install kaggle

In [0]:
!mkdir .kaggle

In [0]:
import json
token = {"username":'nitingrover425','key':'c22685e02df7d46edd199e441980c448'}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!kaggle config set -n path -v{/content}

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

In [ ]:
!kaggle competitions download -c covid19-global-forecasting-week-1 -p /content

In [ ]:
train_data = pd.read_csv("../input/covid19-global-forecasting-week-1/train.csv")

In [ ]:
train_data.head(10)

In [ ]:
train_data.tail(10)

In [ ]:
train_data.shape

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data = train_data.rename(columns = {"Province/State":"State" , "Country/Region":"Country" } )

In [ ]:
train_data.columns

In [ ]:
train_data = train_data.fillna("Not Available")
train_data.isnull().sum()

In [ ]:
train_data.head(10)

In [ ]:
train_data[train_data['ConfirmedCases']<0]

In [ ]:
train_data[train_data['Fatalities']<0]

In [ ]:
train_data['Date'] = pd.to_datetime(train_data['Date'])

In [ ]:
train_data[train_data['ConfirmedCases'] == train_data['ConfirmedCases'].max()]

In [ ]:
train_data['ConfirmedCases'].max()

In [ ]:
train_data['Date'].max()

In [ ]:
date_country_total = train_data.groupby(['Date']).sum()


In [ ]:
date_country_total.head(10)

In [ ]:
date_country_total['Total Cases'] = date_country_total['ConfirmedCases'].cumsum() 

In [ ]:
date_country_total.tail()

In [ ]:
date_country_total.reset_index(inplace = True)
date_country_total.columns

In [ ]:
px.line(date_country_total,x = 'Date', y = 'ConfirmedCases' , title = 'COVID-19 Cases:World')

In [ ]:
px.line(date_country_total.tail() , x = 'Date' , y = 'ConfirmedCases' , title = 'COVID-19 Cases:World[Past 5 days]')

In [ ]:
country_cases = train_data.groupby(['Country' , 'Lat' , 'Long']).sum()
country_cases.reset_index(inplace=True)
country_cases

In [ ]:
country_cases.ConfirmedCases.dtype
country_cases.reset_index(inplace=True)
country_cases.head()

In [ ]:
folium_map = folium.Map(location=[20.5936832, 78.962883],
                            zoom_start=1.5,
                            tiles="CartoDB dark_matter"
                            )

for index,row in country_cases.iterrows():
  radius_confirmed = row['ConfirmedCases']/1000

  color = '#ffd700' #gold #confirmed
  #radius = radius_confirmed

  radius_fatal = row['Fatalities']/1000

  color_f = '#ff0000'  #red #death
  #radius = radius_fatal

  folium.CircleMarker(location=(row['Lat'],row['Long']),radius=radius_confirmed,color=color,fill=True ).add_to(folium_map)
  folium.CircleMarker(location=(row['Lat'],row['Long']),radius=radius_fatal,color=color_f,fill=True ).add_to(folium_map)

  
  

In [ ]:
folium_map

In [ ]:
columns = train_data.columns.tolist()
columns
train_data['Date'] = pd.to_numeric(train_data['Date'])

In [ ]:
columns = [c for c in columns if c not in['Id','ConfirmedCases','Fatalities','State','Country']]
X_train = train_data[columns]
Y1_train = train_data['ConfirmedCases']
Y2_train = train_data['Fatalities']
print(X_train.shape)
print(Y1_train.shape)
print(Y2_train.shape)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 100 ,random_state=0)
regressor.fit(X_train,Y1_train)
#regressor.fit(X_train,Y2_train)

In [ ]:
test_data = pd.read_csv('../input/covid19-global-forecasting-week-1/test.csv')

In [ ]:
test_data.head()

In [ ]:
columnst = test_data.columns.tolist()
columnst
test_data['Date'] = pd.to_datetime(test_data['Date'])

In [ ]:
test_data['Date'] = pd.to_numeric(test_data['Date'])

In [ ]:
columnst = [c for c in columnst if c not in ['ForecastId', 'Province/State', 'Country/Region']]
columnst

In [ ]:
y1_pred = regressor.predict(test_data[columnst])

In [ ]:
regressor.fit(X_train,Y2_train)


In [ ]:
y2_pred = regressor.predict(test_data[columnst])

In [ ]:
pred1 = pd.DataFrame(y1_pred)
pred2 = pd.DataFrame(y2_pred)
sub_df = pd.read_csv('../input/covid19-global-forecasting-week-1/submission.csv')
sub_df.head()

In [ ]:
datasets = pd.concat([sub_df['ForecastId'],pred1,pred2],axis=1)
datasets.columns = ['ForecastId','ConfirmedCases','Fatalities']
datasets.to_csv('submission.csv',index=False)